In [ ]:
#---------------------------
# 베이스라인 만들기
#---------------------------


In [3]:
# 판다스 라이브러리
import pandas as pd

# 데이터 불러오기
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165029,165029,15667085,Meng,667,Spain,Female,33.0,2,0.00,1,1.0,1.0,131834.75,0
165030,165030,15665521,Okechukwu,792,France,Male,35.0,3,0.00,1,0.0,0.0,131834.45,0
165031,165031,15664752,Hsia,565,France,Male,31.0,5,0.00,1,1.0,1.0,127429.56,0
165032,165032,15689614,Hsiung,554,Spain,Female,30.0,7,161533.00,1,0.0,1.0,71173.03,0


In [14]:
# #---------------------------------------
# # EDA
# #---------------------------------------
# # 데이터 샘플
# train.head()
# train.tail(n)
# train.sample(n)
# # 데이터 크기
# train.shape, test.shape
# # 자료형 (타입)
# train.info()
# # 수치형 컬럼 통계값 확인(train)
# train.describe()
# # 수치형 컬럼 통계값 확인(test)
# test.describe()
# # 범주형 컬럼 통계값 확인 (train)
# train.describe(include='O')
# # 범주형 컬럼 통계값 확인 (test)
# test.describe(include='object')
# # 결측치 (train)
# train.isnull().sum()
# # 결측치 (test)
# test.isnull().sum()
# # label(target)별 개수 확인
# train['Exited'].value_counts()

Exited
0    130113
1     34921
Name: count, dtype: int64

In [6]:
#---------------------------------------
# 데이터 전처리
#---------------------------------------
#---------------------
# 결측치 처리 - x
#---------------------

#---------------------
# 이상치 - x
#---------------------

#---------------------
# 인코딩
#---------------------
# 라벨(정답,y값) 분리
y_train = train.pop("Exited")

# 레이블 인코딩

from sklearn.preprocessing import LabelEncoder

cols = train.columns[train.dtypes == object]

for col in cols:
    le = LabelEncoder()
    le.fit(pd.concat([train[col],test[col]] ,axis=0)) # 행합치기
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])



KeyError: 'Exited'

In [9]:
#----------
# y 인코딩
#----------
y_train.value_counts()

Exited
0    130113
1     34921
Name: count, dtype: int64

In [ ]:
#---------------------------------------
# 검증 데이터 분할
#---------------------------------------

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train,
                                                  y_train,
                                                  test_size=0.2,
                                                  random_state=0)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((132027, 13), (33007, 13), (132027,), (33007,))

In [ ]:
#---------------------------------------
# 모델 학습
#---------------------------------------

In [15]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred1=rf.predict_proba(X_val)  
pred1_label=rf.predict(X_val)  

# LightGBM
import lightgbm as lgb
lgbmc = lgb.LGBMClassifier(random_state=0, verbose=-1)
lgbmc.fit(X_train, y_train)
pred2 = lgbmc.predict_proba(X_val)
pred2_label=rf.predict(X_val)  

In [13]:
print(pred1[:,1])
print()
print(pred2[:,1])

[0.11 0.21 0.75 ... 0.28 0.05 0.36]

[0.06442807 0.14527317 0.68723293 ... 0.23911734 0.02022277 0.30957594]


In [ ]:
#---------------------------------------
# 성능 평가
#---------------------------------------

In [17]:
#-------------------
# 랜덤포레스트 성능수치
#-------------------
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val, pred1[:,1])
print('roc_auc:', roc_auc)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, pred1_label)
print('accuracy_score:', accuracy)

from sklearn.metrics import f1_score
f1 = f1_score(y_val, pred1_label, pos_label=1)
print('f1_score:', f1)

roc_auc: 0.8801463308572731
accuracy_score: 0.8627260884054897
f1_score: 0.6168935486598461


In [19]:
#-------------------
# LIGHTGBM 성능수치
#-------------------
from sklearn.metrics import roc_auc_score
roc_auc2_lgm = roc_auc_score(y_val, pred2[:,1])
print('roc_auc:', roc_auc2_lgm)

from sklearn.metrics import accuracy_score
accuracy_lgm = accuracy_score(y_val, pred2_label)
print('accuracy_score:', accuracy_lgm)

from sklearn.metrics import f1_score
f1_lgm = f1_score(y_val, pred2_label, pos_label=1)
print('f1_score:', f1_lgm)

roc_auc: 0.8902162094130004
accuracy_score: 0.8627260884054897
f1_score: 0.6168935486598461


In [ ]:
#---------------------------------------
# 파일로 저장
#---------------------------------------

In [22]:
pred = lgbmc.predict_proba(test)
# pred

# lgbmc.classes_

submit = pd.DataFrame({'pred':pred[:,1]})
submit.to_csv("result.csv", index=False)

pd.read_csv("result.csv")

,pred
0,0.024092
1,0.814112
2,0.027926
3,0.220899
4,0.341188
...,...
110018,0.036409
110019,0.165881
110020,0.017981
110021,0.171473
